## Header

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import SymLogNorm
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.precision = None  # set as needed
    display.float_format = '{:,.8f}'.format

## Data Import

Importing data to pandas dataframe

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]

df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if 'flux' in data:
        df = pd.concat([df,data])
    
display(df.set_index('key').sort_index(ascending=False))

In [ ]:
keys = ['1700658946','1710239438','1710322977','1710355824','1710388814']

In [ ]:
fluxes = [ ]
DOS = [ ]
Emins = [ ]
Emaxs = [ ]

for key in keys:

    # parameter
    tex, sys_size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
    t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
    shift, mag = df.set_index('key').at[key,'shift'], df.set_index('key').at[key,'mag'],
    n_energies, n_moments, n_random_states = df.set_index('key').at[key,'n_energies'], df.set_index('key').at[key,'n_moments'], df.set_index('key').at[key,'n_random_states'],
    scale, epsilon = 12, 0.01,
    fluxes.append(df.set_index('key').at[key,'flux'])

    # data
    qs = np.array([i/sys_size for i in range(1,sys_size//2)])

    emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

    n_q = len(qs)
    dos = []
    for i in range(n_q):
        dos.append( np.load('./data/'+key+'/dos_'+str(i).zfill(4)+'.npy') )
    dos = np.array(dos)
    
    # double range
    qs = np.concatenate((qs,np.array([1-q for q in qs[::-1]])))
    dos = np.concatenate((dos,dos[::-1]))
    
    DOS.append(dos)

    Emins.append(np.amin(emesh))
    Emaxs.append(np.amax(emesh))

    thetamin = np.amin(qs)
    thetamax = np.amax(qs)

## Plot

In [ ]:
set_pandas_display_options()
mpl.pyplot.rcdefaults()
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'CMU serif'
tfs    = 30 #title font size
lfs    = 24 #label font size
fs     = 20 #font size
cbarfs = 20 #colorbar font size

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]

x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(DOS[0].T, aspect='auto',norm=norm, extent=(0,1, Emins[0], Emaxs[0]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim((-11,8))

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

plt.tight_layout()
# plt.savefig("./plots/skx_DoS_kpm.png", dpi=150, bbox_inches = 'tight')

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]

x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(DOS[1].T, aspect='auto',norm=norm, extent=(0,1, Emins[1], Emaxs[1]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim((-11,8))

# ax.axvline(x = 1-fluxes[1], color = 'red', linestyle='--',label = r'$1-\phi$')
# ax.annotate(r'$1-\phi$',(1-fluxes[1]-0.15,1),color='red',fontsize=fs)

# ax.axvline(x = fluxes[1], color = 'red', linestyle='--',label = r'$\phi$')
# ax.annotate(r'$\phi$',(fluxes[1]+0.01,-1.5),color='red',fontsize=fs)

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

plt.tight_layout()
# plt.savefig("./plots/skx_DoS_kpm_flux_37-400.png", dpi=150, bbox_inches = 'tight')

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]

x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(DOS[2].T, aspect='auto',norm=norm, extent=(0,1, Emins[2], Emaxs[2]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim((-11,8))

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

plt.tight_layout()
# plt.savefig("./plots/skx_DoS_kpm_flux_71-400.png", dpi=150, bbox_inches = 'tight')

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]

x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(DOS[3].T, aspect='auto',norm=norm, extent=(0,1, Emins[3], Emaxs[3]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim((-11,8))

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

plt.tight_layout()
# plt.savefig("./plots/skx_DoS_kpm_flux_107-400.png", dpi=150, bbox_inches = 'tight')

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]

x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(DOS[4].T, aspect='auto',norm=norm, extent=(0,1, Emins[4], Emaxs[4]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim((-11,8))

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

plt.tight_layout()
# plt.savefig("./plots/skx_DoS_kpm_flux_151-400.png", dpi=150, bbox_inches = 'tight')

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]

x_ticks = np.arange(0.1, 1.0, 0.2)
y_ticks = np.arange(-10, 10, 2.5)

fig = plt.figure()
gs = fig.add_gridspec(2,60, hspace=0.12,wspace=0)
ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=20))
ax2 = plt.subplot(gs.new_subplotspec((0, 20), colspan=20),sharey=ax1)
ax3 = plt.subplot(gs.new_subplotspec((0, 40), colspan=20),sharey=ax1)
ax4 = plt.subplot(gs.new_subplotspec((1, 10), colspan=20))
ax5 = plt.subplot(gs.new_subplotspec((1, 30), colspan=20),sharey=ax4)
ax6 = plt.subplot(gs.new_subplotspec((1, 52), colspan=1))

col1 = ax1.imshow(DOS[0].T, aspect='auto',norm=norm, extent=(0,1, Emins[0], Emaxs[0]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax1.set_title(r'$\phi=0$',fontsize=fs,y=0.91)
# ax1.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax1.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax1.set_xticks(x_ticks)
ax1.set_yticks(y_ticks)
ax1.tick_params(axis='both', which='major', labelsize=fs)
ax1.set_ylim((-11,9))

# cbar1 = plt.colorbar(col1,ax=(ax1,ax2,ax3),shrink=0.95)

col2 = ax2.imshow(DOS[1].T, aspect='auto',norm=norm, extent=(0,1, Emins[1], Emaxs[1]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax2.set_title(r'$\phi=37/400$',fontsize=fs,y=0.91)
# ax2.set_xlabel(r"$\vartheta$",fontsize=lfs)
# ax2.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax2.set_xticks(x_ticks)
ax2.set_yticks(y_ticks)
ax2.tick_params(axis='both', which='major', labelsize=fs)
ax2.set_ylim((-11,9.5))
ax2.get_yaxis().set_visible(False)

col3 = ax3.imshow(DOS[2].T, aspect='auto',norm=norm, extent=(0,1, Emins[2], Emaxs[2]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax3.set_title(r'$\phi=71/400$',fontsize=fs,y=0.91)
# ax3.set_xlabel(r"$\vartheta$",fontsize=lfs)
# ax3.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax3.set_xticks(x_ticks)
ax3.set_yticks(y_ticks)
ax3.tick_params(axis='both', which='major', labelsize=fs)
ax3.set_ylim((-11,9.5))
ax3.get_yaxis().set_visible(False)

col4 = ax4.imshow(DOS[3].T, aspect='auto',norm=norm, extent=(0,1, Emins[3], Emaxs[3]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax4.set_title(r'$\phi=107/400$',fontsize=fs,y=0.91)
ax4.set_xlabel(r"$\vartheta$",fontsize=lfs)
ax4.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax4.set_xticks(x_ticks)
ax4.set_yticks(y_ticks)
ax4.tick_params(axis='both', which='major', labelsize=fs)
ax4.set_ylim((-11,9.5))

col5 = ax5.imshow(DOS[4].T, aspect='auto',norm=norm, extent=(0,1, Emins[4], Emaxs[4]), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax5.set_title(r'$\phi=151/400$',fontsize=fs,y=0.91)
ax5.set_xlabel(r"$\vartheta$",fontsize=lfs)
# ax5.set_ylabel(r"$E_F \; / \; \lambda_\mathrm{hop}$",fontsize=lfs,labelpad=-10.0)
ax5.set_xticks(x_ticks)
ax5.set_yticks(y_ticks)
ax5.tick_params(axis='both', which='major', labelsize=fs)
ax5.set_ylim((-11,9.5))
ax5.get_yaxis().set_visible(False)

cbar5 = plt.colorbar(col5,cax=ax6)
cbar5.set_label(label=r'DOS (a.u.)', size=lfs)
cbar5.ax.tick_params(labelsize=fs)

plt.tight_layout()
plt.savefig("./plots/skx_DoS_kpm_flux.png", dpi=300, bbox_inches = 'tight')